In [17]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [18]:
if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 2000)

nf = pd.DataFrame(rate)
#         print('getting data')

nf['date']=pd.to_datetime(nf['time'], unit='s')

nf.set_index(np.arange(len(nf)), inplace = True)
nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


# nf.to_excel('EURUSD.xlsx')

df = nf.iloc[:500]  # First half
df2 = nf.iloc[500:]  # Second half

In [19]:
def get_kc( high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))

    # Calculate True Range as the element-wise maximum of the three components
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/atr_lookback).mean()

    kc_middle = close.ewm(span=kc_lookback).mean()
    kc_upper = kc_middle + multiplier * atr
    kc_lower = kc_middle - multiplier * atr

    return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]

def candle(df):
    '''Return candle colour'''

    if df['close'] - df['open'] > 0:
        return 'G'

    else:
        return 'R'
    
    
def calculate_indicators():

#         print('calc_indicator')
    global df
    length = len(df) - 1

    df.loc[length, 'EMA_8'] = talib.EMA(df['close'], timeperiod=8).iloc[-1]
    df.loc[length, 'EMA_50'] = talib.EMA(df['close'], timeperiod=50).iloc[-1]
    df.loc[length, 'EMA_200'] = talib.EMA(df['close'], timeperiod=200).iloc[-1]
    df.loc[length, 'RSI'] = talib.RSI(df['close'], timeperiod=21).iloc[-1]

    df.loc[length, 'candle'] = candle(df.iloc[-1])

    df.loc[length, 'EMA_20'], df.loc[length, 'upper_keltner'], df.loc[length, 'lower_keltner'] = get_kc(df['high'], df['low'], df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)



def check_body( df):
    #     Calculate 0.7 % of entry price
#     fOR 1H FOREX
        one_percent = df['open']/100 
        percent = one_percent * 0.04
        hammer_per = one_percent * 0.04
        
        max_candle_size = 0.11


    #     Calculate candle size close-open
        if df['candle'] == 'G':
            candle_size = df['close'] - df['open']
            upper_wick = df['high'] - df['close']
            lower_wick = df['open'] - df['low']

        else:
            candle_size = abs(df['close'] - df['open'])
            upper_wick = df['high'] - df['open']
            lower_wick = df['close'] - df['low']

        upper_wick_per = (upper_wick/one_percent) 
        lower_wick_per = (lower_wick/one_percent) 



    #     check for entry candle size and Max candle size 0.11 %
        if ((candle_size >= percent) and candle_size <= (max_candle_size * one_percent) and
         (candle_size > upper_wick) and (candle_size > lower_wick) and
        (upper_wick_per < 0.04) and (lower_wick_per < 0.04)):

            if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                return 1

            elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                return 1

            else:

                return 0

    #     condition for hammmer
    #     lower wick is atleast 50 % of total size 
    #     Green Hammer 
    # need to add max wick of opposite dir
        elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
              (candle_size >= hammer_per) and (candle_size > upper_wick)and
              candle_size <= (max_candle_size * one_percent) and ((upper_wick_per < 0.04))  
             and df['high'] < df['upper_keltner']):

            return 1


#     For Red Hammer
# need to add max wick of opposite dir
        elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
              (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (max_candle_size * one_percent) and 
              ((lower_wick_per < 0.04)) and df['low'] > df['lower_keltner'] ):

            return 1


        else:
            return 0


In [20]:
def check_inside_kt( x):

    if  ( x['high'] < x['upper_keltner'] and x['low'] > x['lower_keltner']):
        return 1

    else:
        return 0

In [21]:
def check_last(entry_candle, entry_index, entry_candle_size):
    
    global df
    
#     entry_index = df[df['date'] == entry_candle['date']].index[0]
    
#     entry_candle_size = abs(entry_candle['open'] - entry_candle['close'])

    prev_index = entry_index - 1 
    
    prev_candle = df.iloc[prev_index]
    
    prev_candle_size = abs(prev_candle['open'] - prev_candle['close'])

    if entry_candle['candle'] == 'G':

#       If last candle Red then entry candle must close above last candle open
        if (prev_candle['candle'] == 'R' and entry_candle['close'] >= prev_candle[ 'open'] and
           prev_candle_size >= 0.6 * entry_candle_size):
            
            return True

        else:
            return False

    elif entry_candle['candle'] == 'R':

#       If last candle Red then entry candle must close below last candle open
        if (prev_candle['candle'] == 'G' and entry_candle['close'] <= prev_candle[ 'open'] and
           prev_candle_size >= 0.6 * entry_candle_size):
            
            return True

        else:
            return False
        
    return False



def check_prev_candle(entry_candle):
    
    global df
    

    entry_index = df[df['date'] == entry_candle['date']].index[0]
    entry_candle_size = abs(entry_candle['open'] - entry_candle['close'])
    
    
#     For 2 candle entry
    if check_last(entry_candle, entry_index, entry_candle_size) == True:
        
        return True
    
    else:

        if entry_candle['candle'] == 'G':
            for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
                current_candle = df.iloc[i]
                current_candle_size = abs(current_candle['open'] - current_candle['close'])

                if current_candle['candle'] == 'R':
                    # Calculate the overlap range between the two candles
                    overlap_percentage = ((min(current_candle['open'], entry_candle['close']) - max(current_candle['close'], entry_candle['open'])) /current_candle_size) * 100


                    if (current_candle['candle'] == 'R' and
                        current_candle['open'] <= entry_candle['close'] and
                        current_candle_size >= 0.5 * entry_candle_size and 
                        current_candle_size <= 1.2 * entry_candle_size and
                        overlap_percentage >= 30 ):

                        for j in range(i + 1, entry_index):
                            intermediate_candle = df.iloc[j]
    #                          If intermediate candle goes below 0.02% of red candle no trade
                            if (max(intermediate_candle['open'], intermediate_candle['close']) < current_candle['open'] and
                                j < entry_index - 1 and (current_candle['close'] - current_candle['close']/100 * 0.02) < intermediate_candle['close']):
                                continue

                            elif j == entry_index - 1:
                                return True

                            else:
                                return False
                        break

        elif entry_candle['candle'] == 'R':
    #         print('Testing R candle')
            for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
                current_candle = df.iloc[i]
                current_candle_size = abs(current_candle['open'] - current_candle['close'])

                if current_candle['candle'] == 'G':
                # Calculate the overlap range between the two candles
                    overlap_percentage = ((min(current_candle['open'], entry_candle['close']) - max(current_candle['close'], entry_candle['open'])) /current_candle_size) * 100

    #                     30 % overlap is decided between entry and big opposite candle
                    if ( current_candle['open'] >= entry_candle['close'] and
                        current_candle_size >= 0.5 * entry_candle_size and 
                        current_candle_size <= 1.2 * entry_candle_size and
                        overlap_percentage >= 30):
        #                 print('R in IF')
                        for j in range(i + 1, entry_index):
                            intermediate_candle = df.iloc[j]

                            if (min(intermediate_candle['open'], intermediate_candle['close']) > current_candle['open'] and
                                j < entry_index - 1 and (current_candle['close'] + current_candle['close']/100 * 0.02) > intermediate_candle['close']):
                                continue

                            elif j == entry_index - 1:
                                return True

                            else:
                                return False

                        break

    return False

In [22]:
# Condition Check

checker = pd.DataFrame()

def check_conditions():

    global df, checker
    length = len(df) - 1

#         Checking all entry conditions

    df.loc[length, 'candle'] = candle(df.iloc[-1])
    
    df.loc[length, 'pos'] = check_inside_kt(df.iloc[-1])

    df.loc[length, 'candle_size'] = check_body(df.iloc[-1])
    
#     print('Candle: ',df.loc[length, 'candle'] )
#     print('KT: ', df.loc[length, 'pos'])
#     print('Size: ', df.loc[length, 'candle_size'])

    df.loc[length, 'prev_candle'] = check_prev_candle(df.iloc[-1])
    
#     print('Prev candle: ', df.loc[length, 'prev_candle'])
    
    if (df.loc[length, 'pos'] == 1 and 
        df.loc[length, 'candle_size'] == 1 and df.loc[length, 'prev_candle'] == True):
    
#         print('Complete')
        print('DF: ', df.iloc[-1])


In [23]:
#  Caller
past_df = pd.DataFrame()
def Caller():
    
    global df, df2, past_df
    # Initialize an empty DataFrame to store the past data that gets dropped
    

#         Loop for taking the live data
    for index, row in df2.iterrows():
        # Add live data to the df DataFrame
        df = df.append(row, ignore_index=True)
#             self.df = pd.concat([self.df, row], ignore_index=True)

        # Delete the first row if the DataFrame has more than 202 rows
        if len(df) > 500:
            # Save the dropped row in past_df
            past_df = past_df.append(df.iloc[0], ignore_index=True)
            df = df.iloc[1:].reset_index(drop=True)
            
            
        calculate_indicators()

        check_conditions()

    result_df = pd.concat([past_df, df], ignore_index=True)

    result_df.to_excel('combined.xlsx')

In [24]:
Caller()



C:\Users\hp\AppData\Local\Temp\ipykernel_12068\4103600624.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_12068\4103600624.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_12068\4103600624.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_12068\4103600624.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)


DF:  open                         1.12393
high                          1.1249
low                           1.1239
close                        1.12484
date             2023-07-18 09:45:00
EMA_8                       1.124518
EMA_50                      1.124675
EMA_200                     1.122749
RSI                        50.495108
candle                             G
EMA_20                      1.124821
upper_keltner               1.126972
lower_keltner               1.122669
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.12425
high                          1.1255
low                          1.12419
close                        1.12529
date             2023-07-18 11:45:00
EMA_8                       1.124958
EMA_50                      1.124783
EMA_200                     1.122917
RSI                        52.390236
candle                             G
EMA

C:\Users\hp\AppData\Local\Temp\ipykernel_12068\2714527772.py:57: RuntimeWarning: divide by zero encountered in scalar divide
  overlap_percentage = ((min(current_candle['open'], entry_candle['close']) - max(current_candle['close'], entry_candle['open'])) /current_candle_size) * 100
C:\Users\hp\AppData\Local\Temp\ipykernel_12068\2714527772.py:57: RuntimeWarning: invalid value encountered in scalar divide
  overlap_percentage = ((min(current_candle['open'], entry_candle['close']) - max(current_candle['close'], entry_candle['open'])) /current_candle_size) * 100


DF:  open                         1.12216
high                         1.12287
low                          1.12216
close                        1.12287
date             2023-07-19 01:30:00
EMA_8                       1.122766
EMA_50                      1.122981
EMA_200                     1.122838
RSI                        49.736669
candle                             G
EMA_20                      1.122784
upper_keltner               1.123762
lower_keltner               1.121806
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.11219
high                         1.11319
low                          1.11213
close                        1.11315
date             2023-07-24 05:45:00
EMA_8                        1.11243
EMA_50                      1.112529
EMA_200                     1.114507
RSI                         55.73213
candle                             G
EMA

C:\Users\hp\AppData\Local\Temp\ipykernel_12068\4103600624.py:18: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  past_df = past_df.append(df.iloc[0], ignore_index=True)


DF:  open                         1.10761
high                         1.10762
low                          1.10683
close                        1.10691
date             2023-07-26 09:30:00
EMA_8                       1.107071
EMA_50                      1.105781
EMA_200                     1.106966
RSI                         59.37792
candle                             R
EMA_20                      1.106516
upper_keltner               1.107937
lower_keltner               1.105096
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.11071
high                         1.11083
low                          1.10989
close                        1.10989
date             2023-07-27 02:45:00
EMA_8                       1.109955
EMA_50                      1.108438
EMA_200                     1.107532
RSI                         57.24583
candle                             R
EMA

DF:  open                         1.09899
high                         1.09904
low                          1.09824
close                        1.09833
date             2023-08-10 18:30:00
EMA_8                       1.099192
EMA_50                      1.100636
EMA_200                     1.098997
RSI                        39.770899
candle                             R
EMA_20                       1.10031
upper_keltner               1.102749
lower_keltner               1.097872
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.09831
high                         1.09916
low                          1.09817
close                        1.09904
date             2023-08-10 18:45:00
EMA_8                       1.099158
EMA_50                      1.100573
EMA_200                     1.098998
RSI                        43.247806
candle                             G
EMA

DF:  open                         1.08484
high                         1.08489
low                          1.08403
close                         1.0841
date             2023-09-01 11:30:00
EMA_8                       1.084564
EMA_50                      1.084718
EMA_200                     1.086093
RSI                        45.284502
candle                             R
EMA_20                       1.08467
upper_keltner               1.085969
lower_keltner                1.08337
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.08328
high                         1.08446
low                          1.08321
close                        1.08428
date             2023-09-01 12:00:00
EMA_8                       1.084279
EMA_50                      1.084647
EMA_200                     1.086048
RSI                        47.858826
candle                             G
EMA